In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Path to the dataset
path = './dataset/train/'
class_names = sorted(os.listdir(path))
num_classes = len(class_names)
# Image size
img_size = (192, 192)  # EfficientNetB0 input size

# Load and preprocess data
labels = []
images = []

for cl in class_names:
    for img in os.listdir(path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        labels.append(label)
        
        image = cv2.imread(path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        images.append(image)

# Convert to numpy arrays
labels = np.asarray(labels)
images = np.asarray(images)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.15, random_state=42)

# Load EfficientNetB0 pre-trained on ImageNet without top layer
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=img_size + (3,))

# Add custom top layers for skin diseases classification
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint('efficientnet_model/model{epoch:02d}')






Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 192, 192, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 192, 192, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 192, 192, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 192, 192, 3)          0         ['normalization[0][0]'

In [2]:
# Train the model

from tensorflow.keras.callbacks import EarlyStopping

custom_early_stopping = EarlyStopping(
    monitor='val_accuracy',
    #baseline=0.80,  # Set the baseline accuracy threshold
    patience=3,
    mode='max',
    verbose=1
)

    
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[custom_early_stopping])

Epoch 1/20


179/179 [==============================] - 228s 1s/step - loss: 1.5949 - accuracy: 0.4450 - val_loss: 1.5114 - val_accuracy: 0.4728
Epoch 2/20
179/179 [==============================] - 166s 928ms/step - loss: 1.2498 - accuracy: 0.5672 - val_loss: 1.4331 - val_accuracy: 0.5005
Epoch 3/20
179/179 [==============================] - 169s 944ms/step - loss: 1.0213 - accuracy: 0.6488 - val_loss: 1.4574 - val_accuracy: 0.4985
Epoch 4/20
179/179 [==============================] - 168s 942ms/step - loss: 0.8235 - accuracy: 0.7185 - val_loss: 1.4283 - val_accuracy: 0.5450
Epoch 5/20
179/179 [==============================] - 167s 933ms/step - loss: 0.6549 - accuracy: 0.7761 - val_loss: 1.4398 - val_accuracy: 0.5678
Epoch 6/20
179/179 [==============================] - 166s 928ms/step - loss: 0.5136 - accuracy: 0.8287 - val_loss: 1.5091 - val_accuracy: 0.5598
Epoch 7/20
179/179 [==============================] - 166s 931ms/step - loss: 0.4018 - accuracy: 0.8680 - val_loss: 1.6406 - 

In [9]:
test_path = 'dataset/test1/'

# Load and preprocess test data
test_labels = []
test_images = []

class_names = sorted(os.listdir(test_path))
num_classes = len(class_names)

print("Number of classes in test dataset:", num_classes)
print("Number of classes expected by the model:", model.output_shape[-1])


for cl in class_names:
    for img in os.listdir(test_path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        test_labels.append(label)
        
        image = cv2.imread(test_path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        test_images.append(image)

# Convert to numpy arrays
test_labels = np.asarray(test_labels)
test_images = np.asarray(test_images)

# Evaluate the model on the test data
test_accuracy = model.evaluate(test_images, test_labels)


Number of classes in test dataset: 9
Number of classes expected by the model: 9
67/67 [==============================] - 88s 1s/step - loss: 1.1305 - accuracy: 0.6785


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [3]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession 
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [4]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

# Path to the dataset
path = 'dataset/train/'
class_names = sorted(os.listdir(path))
num_classes = len(class_names)

# Image size
img_size = (192, 192)  # EfficientNetB0 input size

# Load and preprocess data
labels = []
images = []

for cl in class_names:
    for img in os.listdir(path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        labels.append(label)
        
        image = cv2.imread(path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        images.append(image)

# Convert to numpy arrays
labels = np.asarray(labels)
images = np.asarray(images)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.15, random_state=42)

# Load EfficientNetB0 pre-trained on ImageNet without top layer
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=img_size + (3,))

# Add custom top layers for skin diseases classification
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout layer
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout layer
output = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint('efficientnet_model/model{epoch:02d}', save_best_only=True)

# EarlyStopping callback
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 192, 192, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 192, 192, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 192, 192, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 192, 192, 3)          0         ['normalization[0][0]']

In [6]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[checkpoint_callback])

Epoch 1/20


179/179 [==============================] - ETA: 0s - loss: 1.8846 - accuracy: 0.3323INFO:tensorflow:Assets written to: efficientnet_model\model01\assets


INFO:tensorflow:Assets written to: efficientnet_model\model01\assets


179/179 [==============================] - 287s 1s/step - loss: 1.8846 - accuracy: 0.3323 - val_loss: 1.6566 - val_accuracy: 0.4362
Epoch 2/20
179/179 [==============================] - ETA: 0s - loss: 1.6694 - accuracy: 0.4198INFO:tensorflow:Assets written to: efficientnet_model\model02\assets


INFO:tensorflow:Assets written to: efficientnet_model\model02\assets


179/179 [==============================] - 245s 1s/step - loss: 1.6694 - accuracy: 0.4198 - val_loss: 1.5881 - val_accuracy: 0.4421
Epoch 3/20
179/179 [==============================] - ETA: 0s - loss: 1.5753 - accuracy: 0.4551INFO:tensorflow:Assets written to: efficientnet_model\model03\assets


...

179/179 [==============================] - 145s 811ms/step - loss: 1.0671 - accuracy: 0.6345 - val_loss: 1.3682 - val_accuracy: 0.5490
Epoch 12/20
179/179 [==============================] - 113s 629ms/step - loss: 1.0159 - accuracy: 0.6505 - val_loss: 1.3873 - val_accuracy: 0.5262
Epoch 13/20
179/179 [==============================] - ETA: 0s - loss: 0.9810 - accuracy: 0.6710INFO:tensorflow:Assets written to: efficientnet_model\model13\assets


INFO:tensorflow:Assets written to: efficientnet_model\model13\assets


179/179 [==============================] - 141s 791ms/step - loss: 0.9810 - accuracy: 0.6710 - val_loss: 1.3657 - val_accuracy: 0.5569
Epoch 14/20
179/179 [==============================] - 121s 678ms/step - loss: 0.9334 - accuracy: 0.6722 - val_loss: 1.3739 - val_accuracy: 0.5559
Epoch 15/20
179/179 [==============================] - ETA: 0s - loss: 0.9132 - accuracy: 0.6827INFO:tensorflow:Assets written to: efficientnet_model\model15\assets


INFO:tensorflow:Assets written to: efficientnet_model\model15\assets


179/179 [==============================] - 151s 844ms/step - loss: 0.9132 - accuracy: 0.6827 - val_loss: 1.3571 - val_accuracy: 0.5589
Epoch 16/20
179/179 [==============================] - 120s 670ms/step - loss: 0.8705 - accuracy: 0.7001 - val_loss: 1.3873 - val_accuracy: 0.5480
Epoch 17/20
179/179 [==============================] - 118s 657ms/step - loss: 0.8392 - accuracy: 0.7181 - val_loss: 1.3821 - val_accuracy: 0.5608
Epoch 18/20
179/179 [==============================] - 121s 673ms/step - loss: 0.8188 - accuracy: 0.7143 - val_loss: 1.3697 - val_accuracy: 0.5549
Epoch 19/20
179/179 [==============================] - 121s 677ms/step - loss: 0.7783 - accuracy: 0.7281 - val_loss: 1.4487 - val_accuracy: 0.5648
Epoch 20/20
179/179 [==============================] - 119s 667ms/step - loss: 0.7474 - accuracy: 0.7448 - val_loss: 1.4553 - val_accuracy: 0.5786


In [11]:
model.save("skindiseases.json")

INFO:tensorflow:Assets written to: skindiseases.json\assets


INFO:tensorflow:Assets written to: skindiseases.json\assets


In [8]:
test_path = 'dataset/test1/'

# Load and preprocess test data
test_labels = []
test_images = []

class_names = sorted(os.listdir(test_path))
num_classes = len(class_names)

for cl in class_names:
    for img in os.listdir(test_path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        test_labels.append(label)
        
        image = cv2.imread(test_path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        test_images.append(image)

# Convert to numpy arrays
test_labels = np.asarray(test_labels)
test_images = np.asarray(test_images)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

19/19 [==============================] - 13s 661ms/step - loss: 0.6430 - accuracy: 0.8048
Test Loss: 0.6430
Test Accuracy: 0.8048


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

# Define data directories
train_dir = 'dataset/train/'

# Define image dimensions and batch size
img_height, img_width = 224, 224
batch_size = 32

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,
    validation_split=0.2  # Split data into training and validation sets
)

# Load the entire dataset (including training and validation data)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Specify subset as training data
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Specify subset as validation data
)

# Load pre-trained EfficientNetB0 model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze base model layers
base_model.trainable = False

# Add custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output = Dense(9, activation='softmax')(x)

# Compile the model
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
)

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


Found 540 images belonging to 9 classes.


Found 135 images belonging to 9 classes.


Epoch 1/30
16/16 [==============================] - 30s 1s/step - loss: 2.2630 - accuracy: 0.0886 - val_loss: 2.2027 - val_accuracy: 0.1094
Epoch 2/30
16/16 [==============================] - 20s 1s/step - loss: 2.2366 - accuracy: 0.1181 - val_loss: 2.2019 - val_accuracy: 0.1094
Epoch 3/30
16/16 [==============================] - 30s 2s/step - loss: 2.2511 - accuracy: 0.0984 - val_loss: 2.2021 - val_accuracy: 0.1094
Epoch 4/30
16/16 [==============================] - 15s 963ms/step - loss: 2.2552 - accuracy: 0.0906 - val_loss: 2.1972 - val_accuracy: 0.1172
Epoch 5/30
16/16 [==============================] - 17s 1s/step - loss: 2.2258 - accuracy: 0.1280 - val_loss: 2.2009 - val_accuracy: 0.0938
Epoch 6/30
16/16 [==============================] - 15s 969ms/step - loss: 2.2659 - accuracy: 0.0787 - val_loss: 2.1985 - val_accuracy: 0.1016
Epoch 7/30
16/16 [==============================] - 16s 987ms/step - loss: 2.2300 - accuracy: 0.1181 - val_loss: 2.2028 - val_accuracy: 0.1172
Epoch 8/30
